In [202]:
import pandas as pd

In [203]:
# --- Load data ---
eurostat = pd.read_csv('/Users/weronikadorociak/Documents/LSE/MY498 Capstone Project/Coding/Capstone_Project/Data_cleaning/data/eurostat.csv')
nace = pd.read_csv('data/economic_activity_sector.csv')

In [204]:
# --- Prepare Eurostat aging data ---
selected_cols = ['age', 'sex', 'nace_r2', 'geo', 'TIME_PERIOD', 'OBS_VALUE']
eurostat = eurostat[selected_cols].copy()

In [205]:
eu_countries = [
    'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',
    'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
    'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
    'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
    'Slovenia', 'Spain', 'Sweden'
]
excluded = ['No response', 'Total - all NACE activities', 'Unknown NACE activity']

In [206]:
# Filter relevant observations
eurostat = eurostat[
    eurostat["TIME_PERIOD"].isin([2014, 2024]) &
    (eurostat["age"].isin(["15 years or over", "50 years or over"])) &
    (eurostat["geo"].isin(eu_countries))
]

eurostat = eurostat[~eurostat["nace_r2"].isin(excluded)]
eurostat.columns = ['age', 'gender', 'Economic Activity', 'country', 'year', 'num_of_workers']
eurostat["num_of_workers"] = eurostat["num_of_workers"] * 1000

excluded = ['No response', 'Total - all NACE activities', 'Unknown NACE activity']
eurostat = eurostat[~eurostat["Economic Activity"].isin(excluded)]

eurostat = eurostat[~(eurostat["gender"] == "Total")]
eurostat = eurostat.merge(nace, how='left', on = "Economic Activity")

In [208]:
# Pivot for aging ratios
eurostat = eurostat.pivot_table(
    index=["country", "sector"],
    columns=["age", "year", "gender"],
    values="num_of_workers",
    aggfunc="sum"
)

age                                                   15 years or over  \
year                                                              2014   
gender                                                         Females   
country sector                                                           
Austria Accommodation and food service activities             144200.0   
        Activities of extraterritorial organizations            3800.0   
        Activities of households                                   0.0   
        Administrative and support service activities          63000.0   
        Agriculture, forestry and fishing                      88400.0   
...                                                                ...   
Sweden  Public administration and defence                     166800.0   
        Real estate activities                                 27400.0   
        Transportation and storage                             53400.0   
        Water supply; sewerage, waste management                3200.0   
        Wholesale and retail trade                            246300.0   

age                                                                        \
year                                                                 2024   
gender                                                    Males   Females   
country sector                                                              
Austria Accommodation and food service activities       91100.0  134100.0   
        Activities of extraterritorial organizations        0.0    4300.0   
        Activities of households                            0.0       0.0   
        Administrative and support service activities   55000.0   80100.0   
        Agriculture, forestry and fishing              109900.0   48200.0   
...                                                         ...       ...   
Sweden  Public administration and defence              139900.0  256500.0   
        Real estate activities                          48100.0   39100.0   
        Transportation and storage                     191700.0   51800.0   
        Water supply; sewerage, waste management        16900.0    4500.0   
        Wholesale and retail trade                     309500.0  228100.0   

age                                                              \
year                                                              
gender                                                    Males   
country sector                                                    
Austria Accommodation and food service activities      109200.0   
        Activities of extraterritorial organizations     3100.0   
        Activities of households                            0.0   
        Administrative and support service activities   67100.0   
        Agriculture, forestry and fishing               75500.0   
...                                                         ...   
Sweden  Public administration and defence              169800.0   
        Real estate activities                          53600.0   
        Transportation and storage                     175700.0   
        Water supply; sewerage, waste management        21900.0   
        Wholesale and retail trade                     300200.0   

age                                                   50 years or over  \
year                                                              2014   
gender                                                         Females   
country sector                                                           
Austria Accommodation and food service activities              34000.0   
        Activities of extraterritorial organizations               0.0   
        Activities of households                                   0.0   
        Administrative and support service activities          14800.0   
        Agriculture, forestry and fishing                      44000.0   
...                                                          

In [209]:
# Reset index first
eurostat = eurostat.reset_index()

# Flatten MultiIndex columns: (age_group, year, gender) → "agegroup_year_gender"
eurostat.columns = [
    f"{col[0]}_{col[1]}_{col[2]}" if isinstance(col, tuple) else col
    for col in eurostat.columns
]

# Optional: replace spaces with underscores and strip trailing underscores
eurostat.columns = [col.replace(" ", "_").rstrip("_") for col in eurostat.columns]

In [210]:
eurostat = eurostat[~(eurostat["sector"] == 'Activities of extraterritorial organizations')]
eurostat = eurostat[~(eurostat["sector"] == 'Activities of households')]

In [211]:
eurostat_gender_countries = eurostat.groupby("country")[[
    '15_years_or_over_2014_Females',
    '15_years_or_over_2014_Males',
    '15_years_or_over_2024_Females',
    '15_years_or_over_2024_Males',
    '50_years_or_over_2014_Females',
    '50_years_or_over_2014_Males',
    '50_years_or_over_2024_Females',
    '50_years_or_over_2024_Males'
]].sum()

In [212]:
eurostat_gender_economic_activities = eurostat.groupby("sector")[[
    '15_years_or_over_2014_Females',
    '15_years_or_over_2014_Males',
    '15_years_or_over_2024_Females',
    '15_years_or_over_2024_Males',
    '50_years_or_over_2014_Females',
    '50_years_or_over_2014_Males',
    '50_years_or_over_2024_Females',
    '50_years_or_over_2024_Males'
]].sum()

In [214]:
# Compute gender ratios
eurostat_gender_countries["gender_ratio_2014"] = (eurostat_gender_countries['15_years_or_over_2014_Females'] / \
                                                  (eurostat_gender_countries['15_years_or_over_2014_Males'] + eurostat_gender_countries['15_years_or_over_2014_Females'])) * 100

eurostat_gender_countries["gender_ratio_2024"] = (eurostat_gender_countries['15_years_or_over_2024_Females'] / \
                                                 (eurostat_gender_countries['15_years_or_over_2024_Males'] + eurostat_gender_countries['15_years_or_over_2024_Females'])) * 100

eurostat_gender_countries["gender_ratio_2014_2024"] = eurostat_gender_countries["gender_ratio_2024"] - \
                                                      eurostat_gender_countries["gender_ratio_2014"]

eurostat_gender_countries["female_ratio_2014"] = eurostat_gender_countries['50_years_or_over_2014_Females'] / \
                                                 eurostat_gender_countries['15_years_or_over_2014_Females'] * 100

eurostat_gender_countries["female_ratio_2024"] = eurostat_gender_countries['50_years_or_over_2024_Females'] / \
                                                 eurostat_gender_countries['15_years_or_over_2024_Females'] * 100

eurostat_gender_countries["male_ratio_2014"] = eurostat_gender_countries['50_years_or_over_2014_Males'] / \
                                                 eurostat_gender_countries['15_years_or_over_2014_Males'] * 100

eurostat_gender_countries["male_ratio_2024"] = eurostat_gender_countries['50_years_or_over_2024_Males'] / \
                                                 eurostat_gender_countries['15_years_or_over_2024_Males'] * 100

eurostat_gender_countries["female_diff_2014_2024"] = eurostat_gender_countries["female_ratio_2024"] - \
                                                     eurostat_gender_countries["female_ratio_2014"]

eurostat_gender_countries["male_diff_2014_2024"] = eurostat_gender_countries["male_ratio_2024"] - \
                                                     eurostat_gender_countries["male_ratio_2014"]

In [215]:
eurostat_gender_countries = eurostat_gender_countries.round(2).fillna(0).reset_index()

In [217]:
eurostat_gender_countries.to_csv("/Users/weronikadorociak/Documents/LSE/MY498 Capstone Project/Coding/Capstone_Project/Data_cleaning/data/eurostat_gender_countries.csv", index=False)

In [218]:
# Compute gender ratios
eurostat_gender_economic_activities["gender_ratio_2014"] = (eurostat_gender_economic_activities['15_years_or_over_2014_Females'] / \
                                                  (eurostat_gender_economic_activities['15_years_or_over_2014_Males'] + eurostat_gender_economic_activities['15_years_or_over_2014_Females'])) * 100

eurostat_gender_economic_activities["gender_ratio_2024"] = (eurostat_gender_economic_activities['15_years_or_over_2024_Females'] / \
                                                  (eurostat_gender_economic_activities['15_years_or_over_2024_Males'] + eurostat_gender_economic_activities['15_years_or_over_2024_Females'])) * 100

eurostat_gender_economic_activities["gender_ratio_2014_2024"] = eurostat_gender_economic_activities["gender_ratio_2024"] - \
                                                      eurostat_gender_economic_activities["gender_ratio_2014"]

eurostat_gender_economic_activities["female_ratio_2014"] = eurostat_gender_economic_activities['50_years_or_over_2014_Females'] / \
                                                 eurostat_gender_economic_activities['15_years_or_over_2014_Females'] * 100

eurostat_gender_economic_activities["female_ratio_2024"] = eurostat_gender_economic_activities['50_years_or_over_2024_Females'] / \
                                                 eurostat_gender_economic_activities['15_years_or_over_2024_Females'] * 100

eurostat_gender_economic_activities["male_ratio_2014"] = eurostat_gender_economic_activities['50_years_or_over_2014_Males'] / \
                                                 eurostat_gender_economic_activities['15_years_or_over_2014_Males'] * 100

eurostat_gender_economic_activities["male_ratio_2024"] = eurostat_gender_economic_activities['50_years_or_over_2024_Males'] / \
                                                 eurostat_gender_economic_activities['15_years_or_over_2024_Males'] * 100

eurostat_gender_economic_activities["female_diff_2014_2024"] = eurostat_gender_economic_activities["female_ratio_2024"] - \
                                                     eurostat_gender_economic_activities["female_ratio_2014"]

eurostat_gender_economic_activities["male_diff_2014_2024"] = eurostat_gender_economic_activities["male_ratio_2024"] - \
                                                     eurostat_gender_economic_activities["male_ratio_2014"]

eurostat_gender_economic_activities =  eurostat_gender_economic_activities.round(2).fillna(0).reset_index()

In [219]:
eurostat_gender_economic_activities.to_csv("/Users/weronikadorociak/Documents/LSE/MY498 Capstone Project/Coding/Capstone_Project/Data_cleaning/data/eurostat_gender_economic_activities.csv", index=False)